In [28]:
pip install dash_html_components

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
import plotly.express as px
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input,Output
from app import app


In [ ]:
app = dash.Dash(__name__)


In [ ]:
df = pd.read_csv("Teen rates by state.csv")
df = df.groupby(['STATE','YEAR'])[['RATE']].mean()
df.reset_index(inplace=True)  
print(df[:5])

In [ ]:
TeenBirthRates = df[df.YEAR != 2005]

In [ ]:
print(TeenBirthRates[:5])

In [ ]:
app.layout = html.Div([
    html.H1("Teen Birth Rates in USA", style ={'text-align': 'center'}),
    
    dcc.Dropdown(id ="slct_year",
                options =[
                    {"label": "2014", "value": 2014},
                    {"label": "2015", "value": 2015},
                    {"label": "2016", "value": 2016},
                    {"label": "2017", "value": 2017},
                    {"label": "2018", "value": 2018},
                    {"label": "2019", "value": 2019},
                    {"label": "2020", "value": 2020}],
                 multi = False,
                 value = 2014,
                 style = {'width': "40%"}
                ),
    html.Div(id='output_container', children = []),
    html.Br(),
    
    dcc.Graph(id='My state map', figure={})
                
])

@app.callback(
    [Output(component_id='output_container', component_property='children'),
     Output(component_id='My state map', component_property='figure')],
    [Input(component_id='slct_year', component_property='value')]
)
def update_graph(option_slctd):
    print(option_slctd)
    print(type(option_slctd))

    container = "The year chosen by user was: {}".format(option_slctd)

    dff = TeenBirthRates.copy()
    dff = dff[dff["YEAR"] == option_slctd]
    dff = dff[dff["STATE"] == "TX"]

    # Plotly Express
    fig = px.choropleth(
        data_frame=dff,
        locationmode='USA-states',
        locations='state_code',
        scope="usa",
        color='STATE',
        hover_data=['State', 'RATES'],
        color_continuous_scale=px.colors.sequential.YlOrRd,
        labels={'Pct of Colonies Impacted': '% of Bee Colonies'},
        template='plotly_dark'
    )

    return container,fig


In [ ]:
if __name__ == '__main__':
    app.run_server(
    port=8050,
    host='0.0.0.0'
    )